In [21]:
import random
from datetime import datetime, timedelta

# List of creative dream descriptions
dream_descriptions = [
    "Flying over a magical city with colorful buildings",
    "Exploring a mysterious underwater world with glowing sea creatures",
    "Walking through a forest of talking trees under a starlit sky",
    "Attending a grand masquerade ball in a castle from a bygone era",
    "Meeting my childhood hero in a whimsical dreamland",
    "Solving a complex puzzle in a room filled with floating objects",
    "Dancing under the shimmering lights of a celestial aurora",
    "Discovering a hidden portal to a parallel universe",
    "Riding a unicorn through a field of sparkling flowers",
    "Conversing with animals in an enchanted forest glade"
]

# Generate synthetic dream data for 50 people with unique dream descriptions
NUM_PEOPLE = 50
MIN_DAYS = 10
dream_data = []

for person_id in range(1, NUM_PEOPLE + 1):
    unique_dreams = random.sample(dream_descriptions, MIN_DAYS)  # Unique dreams for each person
    for day, dream_description in enumerate(unique_dreams, start=1):
        for _ in range(random.randint(1, 3)):  # Generate random number of dreams per day
            dream_date = datetime.now() - timedelta(days=day)  # Generate dream date
            dream_data.append({
                'user_id': person_id,
                'dream_date': dream_date.strftime('%Y-%m-%d'),
                'dream_description': dream_description
            })

# Print the generated dream data for all 50 people with unique dream descriptions
for dream in dream_data:
    print(f"User ID: {dream['user_id']} | Dream Date: {dream['dream_date']}  | Dream Description: {dream['dream_description']}")


User ID: 1 | Dream Date: 2024-07-04  | Dream Description: Conversing with animals in an enchanted forest glade
User ID: 1 | Dream Date: 2024-07-04  | Dream Description: Conversing with animals in an enchanted forest glade
User ID: 1 | Dream Date: 2024-07-04  | Dream Description: Conversing with animals in an enchanted forest glade
User ID: 1 | Dream Date: 2024-07-03  | Dream Description: Walking through a forest of talking trees under a starlit sky
User ID: 1 | Dream Date: 2024-07-03  | Dream Description: Walking through a forest of talking trees under a starlit sky
User ID: 1 | Dream Date: 2024-07-02  | Dream Description: Exploring a mysterious underwater world with glowing sea creatures
User ID: 1 | Dream Date: 2024-07-01  | Dream Description: Solving a complex puzzle in a room filled with floating objects
User ID: 1 | Dream Date: 2024-06-30  | Dream Description: Discovering a hidden portal to a parallel universe
User ID: 1 | Dream Date: 2024-06-29  | Dream Description: Attending a g

In [23]:
import re
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Define a function for text cleaning
def clean_text(text):
    text = text.lower()  # Convert text to lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters and numbers
    return text

# Clean and tokenize the dream descriptions
cleaned_dreams = [clean_text(dream['dream_description']) for dream in dream_data]

# Tokenize the cleaned dream descriptions
tokenizer = Tokenizer()
tokenizer.fit_on_texts(cleaned_dreams)
sequences = tokenizer.texts_to_sequences(cleaned_dreams)

# Generate input sequences for the LSTM model
max_sequence_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post')

# Print the tokenized sequences
for i, sequence in enumerate(padded_sequences):
    print(f"Sequence {i + 1}: {sequence}")

# Print the word index mapping from the tokenizer
print("\nWord Index Mapping:")
print(tokenizer.word_index)

Sequence 1: [40  3 41  2 42 43  6 44  0  0  0  0]
Sequence 2: [40  3 41  2 42 43  6 44  0  0  0  0]
Sequence 3: [40  3 41  2 42 43  6 44  0  0  0  0]
Sequence 4: [29  7  1  6  4 30 31  5  1 32 33  0]
Sequence 5: [29  7  1  6  4 30 31  5  1 32 33  0]
Sequence 6: [45  1 46 47 48  3 49 50 51  0  0  0]
Sequence 7: [16  1 17 18  2  1 19 20  3 21 22  0]
Sequence 8: [52  1 53 54 55  1 56 57  0  0  0  0]
Sequence 9: [ 8  1  9 10 11  2  1 12 13  1 14 15]
Sequence 10: [ 8  1  9 10 11  2  1 12 13  1 14 15]
Sequence 11: [ 8  1  9 10 11  2  1 12 13  1 14 15]
Sequence 12: [34  5 35 36 37  4  1 38 39  0  0  0]
Sequence 13: [23 24 25 26  2  1 27 28  0  0  0  0]
Sequence 14: [23 24 25 26  2  1 27 28  0  0  0  0]
Sequence 15: [23 24 25 26  2  1 27 28  0  0  0  0]
Sequence 16: [58 59  1 60 61  3 62 63  0  0  0  0]
Sequence 17: [58 59  1 60 61  3 62 63  0  0  0  0]
Sequence 18: [58 59  1 60 61  3 62 63  0  0  0  0]
Sequence 19: [64  1 65  7  1 66  4 67 68  0  0  0]
Sequence 20: [52  1 53 54 55  1 56 57  0

In [25]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, LeakyReLU, Input, Reshape, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

def build_generator(vocab_size, max_sequence_length):
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size, output_dim=100, input_length=max_sequence_length))
    model.add(LSTM(256, return_sequences=True))
    model.add(LSTM(256))
    model.add(Dense(max_sequence_length, activation='softmax'))  # Use softmax for output probabilities
    return model

def build_discriminator(max_sequence_length):
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size, output_dim=100, input_length=max_sequence_length))  # Use vocab_size
    model.add(LSTM(256, return_sequences=True))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    return model

def build_gan(generator, discriminator):
    discriminator.trainable = True  # Set discriminator trainable to True before compiling

    discriminator.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy', metrics=['binary_accuracy'])
    
    gan_input = Input(shape=(max_sequence_length,))
    generated_dream = generator(gan_input)
    gan_output = discriminator(generated_dream)
    
    gan = Model(gan_input, gan_output)
    gan.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy', metrics=['binary_accuracy'])
    return gan

vocab_size = len(tokenizer.word_index) + 1
generator = build_generator(vocab_size, max_sequence_length)
discriminator = build_discriminator(max_sequence_length)
gan = build_gan(generator, discriminator)


In [27]:
import numpy as np
def train_gan(generator, discriminator, gan, data, epochs, batch_size):
    for epoch in range(epochs):
        idx = np.random.randint(0, data.shape[0], batch_size)
        real_dreams = data[idx]
        noise = np.random.normal(0, 1, (batch_size, max_sequence_length))
        
        # Train the discriminator
        d_loss_real = discriminator.train_on_batch(real_dreams, np.ones((batch_size, 1)))
        generated_dreams = generator.predict(noise)
        d_loss_fake = discriminator.train_on_batch(generated_dreams, np.zeros((batch_size, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        # Train the generator
        noise = np.random.normal(0, 1, (batch_size, max_sequence_length))
        g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))
        
        if epoch % 10 == 0:
            print(f"{epoch}/{epochs} [D loss: {d_loss[0]} | D accuracy: {100 * d_loss[1]}] [G loss: {g_loss}]")

# Train the GAN
train_gan(generator, discriminator, gan, padded_sequences, epochs=10, batch_size=32)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step
0/10 [D loss: 0.6915262937545776 | D accuracy: 58.59375] [G loss: [array(0.6922462, dtype=float32), array(0.6922462, dtype=float32), array(0., dtype=float32), array(0., dtype=float32)]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


In [29]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Prepare the input and target data for training
tokenizer = Tokenizer()
tokenizer.fit_on_texts([dream['dream_description'] for dream in dream_data])
sequences = tokenizer.texts_to_sequences([dream['dream_description'] for dream in dream_data])
padded_sequences = pad_sequences(sequences, padding='post')

X = padded_sequences[:, :-1]  # Input sequences
Y = padded_sequences[:, 1:]   # Target sequences

# Define the LSTM model architecture
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=100))  
model.add(LSTM(128, return_sequences=True))
model.add(Dense(len(tokenizer.word_index)+1, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the LSTM model
model.fit(X, Y, epochs=20, batch_size=32, verbose=1)

# Save the trained model
model.save('dream_analysis_lstm_model.h5')

# Evaluate the model on the training data
loss, accuracy = model.evaluate(X, Y)
print(f'Training Accuracy: {accuracy * 100:.2f}%')

Epoch 1/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.2417 - loss: 3.9526
Epoch 2/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3423 - loss: 3.0362
Epoch 3/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4073 - loss: 2.6013
Epoch 4/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5499 - loss: 1.7866
Epoch 5/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8282 - loss: 0.9801
Epoch 6/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9416 - loss: 0.5100
Epoch 7/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9629 - loss: 0.2870
Epoch 8/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9806 - loss: 0.1723
Epoch 9/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 0.1042
Epoch 10/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 0.0643
Epoch 11/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 0.0433
Epoch 12/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - lo

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.0064
Training Accuracy: 100.00%


In [19]:
import random
import re
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import load_model

# Assuming dream_data is available and model is saved as 'dream_analysis_lstm_model.h5'

# Load the trained model
model = load_model('dream_analysis_lstm_model.h5')

# Tokenizer setup
tokenizer = Tokenizer()
tokenizer.fit_on_texts([dream['dream_description'] for dream in dream_data])
max_sequence_length = max([len(seq) for seq in tokenizer.texts_to_sequences([dream['dream_description'] for dream in dream_data])])

# Define a function for text cleaning
def clean_text(text):
    text = text.lower()  # Convert text to lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters and numbers
    return text

# Define a function to get user input
def get_user_input():
    user_id = int(input("Enter your user ID: "))
    dream_date = input("Enter the dream date (YYYY-MM-DD): ")
    dream_description = input("Enter your dream description: ")
    return user_id, dream_date, dream_description

# Define a function to predict the future dream
def predict_future_dream(dream_description):
    # Clean and tokenize the dream description
    cleaned_dream = clean_text(dream_description)
    sequence = tokenizer.texts_to_sequences([cleaned_dream])
    padded_sequence = pad_sequences(sequence, maxlen=max_sequence_length, padding='post')
    
    # Predict the next sequence of the dream
    prediction = model.predict(padded_sequence)
    predicted_word_index = np.argmax(prediction, axis=-1)
    
    # Map the word index to the actual word
    index_to_word = {index: word for word, index in tokenizer.word_index.items()}
    predicted_words = [index_to_word.get(index, '') for index in predicted_word_index[0]]
    predicted_dream = ' '.join(predicted_words)
    
    return predicted_dream

# Define a function for personalized visualization
def visualize_dreams(user_id):
    user_dreams = [dream for dream in dream_data if dream['user_id'] == user_id]
    
    # Extract dream dates and descriptions
    dream_dates = [datetime.strptime(dream['dream_date'], '%Y-%m-%d') for dream in user_dreams]
    dream_descriptions = [dream['dream_description'] for dream in user_dreams]


# Main function to run the analysis and visualization
def main():
    user_id, dream_date, dream_description = get_user_input()
    predicted_dream = predict_future_dream(dream_description)
    print(f'Predicted future dream: {predicted_dream}')
    
    visualize_dreams(user_id)

# Run the main function
if __name__ == "__main__":
    main()


Enter your user ID:  22
Enter the dream date (YYYY-MM-DD):  2024-09-09
Enter your dream description:  Exploring a mysterious underwater world with glowing sea creatures


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
Predicted future dream: a mysterious underwater world with glowing sea creatures    
